<center>
    <h1>Cloud Movement Diffusion</h1>
    <h2>Artificial Intelligence in Industry</h2>
    <h3>Facundo Nicolas Maidana, Farshid Mahmoudabadi, and Riccardo Spolaor</h3>
    <h4>riccardo.spolaor@studio.unibo.it</h4>
</center>

---

This project explores the use of diffusion models for short-term solar energy forecasting. Diffusion models are a type of generative model that can be used to predict the future state of a system by gradually "diffusing" from the current state. In this project, we train a diffusion model on a dataset of cloud images. We then use the model to predict the future state of the clouds, which can be used to forecast solar energy production.

Introduction:

Solar energy is a promising renewable energy source, but it is intermittent. This means that the amount of solar energy produced can vary significantly from day to day. Short-term solar energy forecasting can help to mitigate this variability by providing estimates of solar energy production in the near future.

Diffusion models:

Diffusion models are a type of generative model that can be used to predict the future state of a system by gradually "diffusing" from the current state. Diffusion models are trained on a dataset of images, and they can be used to generate new images that are similar to the images in the dataset.

Project goals:

The goals of this project are to:

Train a diffusion model on a dataset of cloud images.
Use the model to predict the future state of the clouds.
Use the predicted cloud state to forecast solar energy production.

Project results:

The results of this project show that diffusion models can be used to predict the future state of the clouds with some accuracy. The predicted cloud state can then be used to forecast solar energy production.

# Settings

In [1]:
# Settings for autoreloading.

%load_ext autoreload
%autoreload 2

In [2]:
!pip install -q wandb tqdm matplotlib diffusers fastcore fastprogress torchmetrics

In [3]:
from types import SimpleNamespace

# TODO: Some of these parameters are not used, remove them.
config = SimpleNamespace(
    epochs=101, # Number of epochs.
    model_name='unet_small', # Model name to save [unet_small, unet_big].
    strategy='ddpm', # Strategy to use ddpm.
    noise_steps=1_000, # Number of noise steps on the diffusion process.
    sampler_steps=333, # Number of sampler steps on the diffusion process.
    seed=42, # Random seed.
    batch_size=128, # Batch size.
    img_size=64, # Image size.
    device='cuda', # Device to use.
    num_workers=0, # Number of workers for dataloader.
    num_input_frames=3, # Number of frames to use as input for the training.
    lr=5e-4, # Learning rate.
    log_every_epoch=5, # Log every n epochs to wandb.
    num_prediction_frames=1, # Number of predictions to make.
    num_channels=1, # Number of channels.
    validate_epochs=False, # Whether to validate every epoch.
    )

In [4]:
import wandb
wandb.login(key='d3d31786e16c5dffe9ee01690d0bb069cac55e84')

wandb: Currently logged in as: riccardo-spolaor94 (ai-industry). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## training

In [5]:
PROJECT_NAME = 'sevir'
DATASET_ARTIFACT = 'ai-industry/sevir/SEVIR:latest'
SCALERS_ARTIFACT = 'ai-industry/sevir/scalers:latest'

In [6]:
DATASET_ARTIFACT

'ai-industry/sevir/SEVIR:latest'

In [7]:
# TODO: repeated code just do the scaling here.
import numpy as np

class Scaler():
    def __init__(self, dataset: np.ndarray) -> None:
        self.min = dataset.min()
        self.max = dataset.max()

    def scale(self, array: np.ndarray) -> np.ndarray:
        return .5 - (array - self.min) / (self.max - self.min)

    def unscale(self, array: np.ndarray) -> np.ndarray:
        return ((.5 - array) * (self.max - self.min)) + self.min

In [8]:
import torch

from src.seeder import set_seed
from src.model import get_unet_params, add_gru_params
from src.dataset_builder import download_dataset, download_scalers

# Set the model parameters in the config file.
'''config.model_params = get_unet_params(
    model_name=config.model_name,
    num_frames=10)''';

# Set the model parameters in the config file.
# TODO: base num_frames on config
config.model_params = get_unet_params(
    model_name=config.model_name,
    num_input_frames=config.num_input_frames + config.num_prediction_frames,
    num_output_frames=config.num_prediction_frames,
    num_channels=config.num_channels)

config.model_params = add_gru_params(
    config.model_params,
    num_input_past_frames=config.num_input_frames,
    num_input_prediction_frames=config.num_prediction_frames,
    num_channels=config.num_channels,
    mode='last_output')

# Set the seed and device.
set_seed(config.seed)
device = torch.device(config.device)

# Downlaod the dataset from the wandb Artifact.
files = download_dataset(DATASET_ARTIFACT, PROJECT_NAME)
scalers = download_scalers(SCALERS_ARTIFACT, PROJECT_NAME)

wandb: Downloading large artifact SEVIR:latest, 8544.38MB. 9 files... 
wandb:   9 of 9 files downloaded.  
Done. 0:0:5.8


wandb:   3 of 3 files downloaded.  


In [9]:
from src.dataset_builder import CloudDataset

# Build the train and validation datasets.
train_ds = CloudDataset(
    files=files[1:2], # Use just ir069 training images.
    num_frames=config.num_input_frames + config.num_prediction_frames, # Use 3 frames as input and 1 frame as prediction.
    img_size=config.img_size, # Image size.
    scalers=scalers[0:1] # Use just ir069 scaler.
    ).shuffle()
valid_ds = CloudDataset(
    files=files[2:3], # Use just ir069 validation images.
    num_frames=config.num_input_frames + 3, # Use 3 frames as input and 3 frames as target.
    img_size=config.img_size, # Image size.
    scalers=scalers[0:1]) # Use just ir069 scaler.

In [10]:
from src.dataloader import NoisifyDataloader, ValidationDataloader
from src.model import UNet2DTemporalCondition
from src.sampler import ddim_sampler
from src.training import MiniTrainer


# Build the DDPM noise train and validation dataloaders.
train_dataloader = NoisifyDataloader(
    train_ds,
    config.batch_size,
    shuffle=True,
    n_frames_to_nosify=config.num_prediction_frames,
    num_workers=config.num_workers)

valid_dataloader = ValidationDataloader(
    valid_ds,
    config.batch_size,
    n_past_frames=config.num_input_frames,
    shuffle=False,
    num_workers=config.num_workers)

# Setup the model.
model = UNet2DTemporalCondition(**config.model_params)

# Setup the sampler.
sampler = ddim_sampler(
    steps=config.sampler_steps,
    n_frames_to_predict=config.num_prediction_frames,
    n_channels=config.num_channels)

# Get the trainer.
trainer = MiniTrainer(
    train_dataloader,
    valid_dataloader,
    model,
    sampler,
    scalers[0],
    device,
    n_frames_to_predict=config.num_prediction_frames,
    n_auto_regression_steps=3)

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `PeakSignalNoiseRatio` from `torchmetrics` was deprecated and will be removed in 2.0. Import `PeakSignalNoiseRatio` from `torchmetrics.image` instead.
  _future_warning(
/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `StructuralSimilarityIndexMeasure` from `torchmetrics` was deprecated and will be removed in 2.0. Import `StructuralSimilarityIndexMeasure` from `torchmetrics.image` instead.
  _future_warning(


In [ ]:
# trainer = MiniTrainer(train_dataloader, valid_dataloader, model, sampler, device)

# Train the model.
with wandb.init(
    project='cloud-diffuser-unet-gru-last-output',
    entity='ai-industry',
    config=config,
    tags=['ddpm', config.model_name]):
    histories = trainer.fit(config)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([128, 3, 64, 64])) that is different to the input size (torch.Size([128, 1, 64, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([96, 3, 64, 64])) that is different to the input size (torch.Size([96, 1, 64, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


torch.Size([64, 3, 64, 64])


Traceback (most recent call last):
  File "<ipython-input-11-84f94028a0f0>", line 9, in <cell line: 4>
    histories = trainer.fit(config)
  File "/content/src/training.py", line 313, in fit
    self.one_epoch_validation(
  File "/content/src/training.py", line 209, in one_epoch_validation
    prediction_frames = prediction_frames.reshape(
RuntimeError: shape '[64, 1, 1, 64, 64]' is invalid for input of size 786432


In [ ]:
from src.visualization import plot_metrics

plot_metrics(
    *histories[:2],
    PSNR=histories[2],
    SSIM=histories[3],
    mCSI=histories[4])